# NTK classifier for Cora

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import numpy as np
from exp_ntk_certify import run
# from exp_ntk_svm_hyperparam import run

Use from seml.experiment import setup_logger instead.
Note that seml.experiment.Experiment already includes the logger setup.
See https://github.com/TUM-DAML/seml/blob/master/examples/example_experiment.py


In [4]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str

In [5]:
seed = 0

data_params = dict(
    dataset = "csbm",
    learning_setting = "inductive", # or "transdructive"
    specification = dict(
        classes = 2,
        n_trn_labeled = 20,
        n_trn_unlabeled = 0,
        n_val = 20,
        n_test = 500,
        sigma = 1,
        avg_within_class_degree = 1.58 * 2,
        avg_between_class_degree = 0.37 * 2,
        K = 1.5,
        seed = 0 # used to generate the dataset & data split
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
)

certificate_params = dict(
    n_adversarial = 10, # number adversarial nodes
    perturbation_model = "l0",
    delta = 0.01 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

# bilevel svm

In [6]:
def validate_bilevel_svm(n_trn_labeled=4, n_val=4, n_test=2, delta=0, n_adversarial=0, C=1):
    seed = 0
    data_params = dict(
    dataset = "csbm",
    learning_setting = "transductive", #or "transdructive"
    specification = dict(
        classes = 2,
        n_trn_labeled = n_trn_labeled, #40, 
        n_trn_unlabeled = 0,
        n_val = n_val, #40,
        n_test = n_test, #1000,
        sigma = 1,
        avg_within_class_degree = 1.58 * 2,
        avg_between_class_degree = 0.37 * 2,
        K = 1.5,
        seed = 10 # used to generate the dataset & data split
    )
    )

    # small graph, C=1, m.Params.IntegralityFocus = 1 , m.Params.IntFeasTol = 1e-8
    model_params["activation"] = "relu"
    model_params["regularizer"] = C
    model_params["pred_method"] = "svm"
    model_params["cache_size"] = 10000
    model_params["bias"] = False
    model_params["solver"] = "qplayer"
    model_params["alpha_tol"] = 1e-4
    data_params["dataset"] = "csbm"
    data_params["specification"]["avg_within_class_degree"] = 1.58 * 2
    data_params["specification"]["avg_between_class_degree"] = 0.37 * 2
    data_params["learning_setting"] = "transductive"
    other_params["device"] = "cpu"
    certificate_params["n_adversarial"] = n_adversarial
    certificate_params["delta"] = delta
    certificate_params["method"] = "XXT"
    certificate_params["perturbation_model"] = "linf"
    certificate_params["cert_method"] = "bilevel_svm"
    verbosity_params["debug_lvl"] = "info"
    data_params["specification"]["seed"] = seed
    other_params["debug"] = True
    result = run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)
    # data_params["cv_folds"] = 2
    # result = run(data_params, model_params, verbosity_params, other_params, seed)
    return result


In [22]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0, n_adversarial=0)

[22:06:51] INFO     Starting experiment            ]8;id=400371;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=280417;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#105\105]8;;\
                    exp_ntk_svm_hyperparam with                                 
                    configuration:                                              

           INFO     data_params: {'dataset':       ]8;id=726734;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=91932;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#106\106]8;;\
                    'csbm', 'learning_setting':                                 
                    'transductive',                                             
                    'specification': {'classes':                                
                    2, 'n_trn_labeled': 40,                                     
                    'n_trn_unlabeled': 0, 'n_val':                              
                    40, 'n_test': 120, 'sigma': 1,                              
                    'avg_within_class_degree':                                  
                    3.16,                                                       
                    'avg_between_class_degree':                                 
                    0.74, 'K': 1.5, 'seed': 0},                                 
                    'cv_folds': 2}                                              

           INFO     model_params: {'label': 'GCN', ]8;id=259395;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=930005;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#107\107]8;;\
                    'model': 'GCN',                                             
                    'normalization':                                            
                    'row_normalization', 'depth':                               
                    1, 'regularizer': 1,                                        
                    'pred_method': 'svm',                                       
                    'activation': 'relu',                                       
                    'cache_size': 10000, 'bias':                                
                    False, 'solver': 'qplayer',                                 
                    'alpha_tol': 0.0001}                                        

           INFO     verbosity_params:              ]8;id=816266;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=678776;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#108\108]8;;\
                    {'debug_lvl': 'info'}                                       

           INFO     other_params: {'device':       ]8;id=85143;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=13541;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#109\109]8;;\
                    'cpu', 'dtype': torch.float64,                              
                    'allow_tf32': False, 'debug':                               
                    True}                                                       

...

           INFO     X_rowsum.median(): -0.23559162765741348          ]8;id=217100;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=703508;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#170\170]8;;\

           INFO     X_rowsum.min(): -8.594079315662384               ]8;id=13881;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=499060;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#171\171]8;;\

           INFO     X_rowsum.max(): 8.860138446092606                ]8;id=680216;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=372917;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#172\172]8;;\

Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Alphas found: ['0.0000', '1.0000', '-0.0000', '0.7567', '-0.0000', '0.0000', '-0.0000', '0.9532', '1.0000', '-0.0000', '-0.0000', '0.0000', '-0.0000', '-0.0000', '0.6292', '-0.0000', '0.9961', '-0.0000', '1.0000', '-0.0000', '0.7446', '-0.0000', '-0.0000', '-0.0000', '1.0000', '0.0000', '0.0000', '-0.0000', '0.9325', '-0.0000', '-0.0000', '-0.0000', '1.0000', '-0.0000', '-0.0000', '1.0000', '-0.0000', '-0.0000', '-0.0000', '0.6247']
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.277894448

{'trn_acc': 0.9375,
 'val_acc_l': [0.875, 0.925000011920929],
 'test_acc': 0.8916666507720947,
 'trn_min_ypred': -4.389232673682621,
 'trn_max_ypred': 2.99784697290841,
 'trn_min_ntkunlabeled': -0.6051960390713326,
 'trn_max_ntkunlabeled': 16.55578889740822,
 'val_svm_alphas': [tensor([ 1.9751e-11,  1.0000e+00, -1.4982e-12,  7.5672e-01, -1.1112e-12,
           4.0568e-13, -1.4902e-12,  9.5317e-01,  1.0000e+00, -4.2858e-13,
          -1.0075e-12,  4.7687e-13, -4.6778e-12, -1.1055e-11,  6.2921e-01,
          -8.6175e-12,  9.9608e-01, -3.8548e-12,  1.0000e+00, -2.1137e-12,
           7.4460e-01, -2.5484e-12, -1.8758e-12, -2.6960e-12,  1.0000e+00,
           1.5644e-12,  5.7693e-13, -9.3825e-13,  9.3254e-01, -4.2501e-13,
          -7.6238e-12, -1.4027e-12,  1.0000e+00, -3.2959e-12, -7.0764e-12,
           1.0000e+00, -6.2521e-12, -9.7921e-12, -3.6837e-12,  6.2467e-01],
         dtype=torch.float64),
  tensor([ 1.0000e+00,  1.0000e+00, -1.3287e-11, -5.7338e-12, -1.2727e-11,
           7.752

In [23]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0, n_adversarial=0, C=0.1)

[22:07:20] INFO     Starting experiment            ]8;id=413520;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=69982;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#105\105]8;;\
                    exp_ntk_svm_hyperparam with                                 
                    configuration:                                              

           INFO     data_params: {'dataset':       ]8;id=123787;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=447613;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#106\106]8;;\
                    'csbm', 'learning_setting':                                 
                    'transductive',                                             
                    'specification': {'classes':                                
                    2, 'n_trn_labeled': 40,                                     
                    'n_trn_unlabeled': 0, 'n_val':                              
                    40, 'n_test': 120, 'sigma': 1,                              
                    'avg_within_class_degree':                                  
                    3.16,                                                       
                    'avg_between_class_degree':                                 
                    0.74, 'K': 1.5, 'seed': 0},                                 
                    'cv_folds': 2}                                              

           INFO     model_params: {'label': 'GCN', ]8;id=893450;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=513466;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#107\107]8;;\
                    'model': 'GCN',                                             
                    'normalization':                                            
                    'row_normalization', 'depth':                               
                    1, 'regularizer': 0.1,                                      
                    'pred_method': 'svm',                                       
                    'activation': 'relu',                                       
                    'cache_size': 10000, 'bias':                                
                    False, 'solver': 'qplayer',                                 
                    'alpha_tol': 0.0001}                                        

           INFO     verbosity_params:              ]8;id=654000;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=442122;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#108\108]8;;\
                    {'debug_lvl': 'info'}                                       

           INFO     other_params: {'device':       ]8;id=943979;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=965703;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#109\109]8;;\
                    'cpu', 'dtype': torch.float64,                              
                    'allow_tf32': False, 'debug':                               
                    True}                                                       

...

           INFO     X_rowsum.median(): -0.23559162765741348          ]8;id=649220;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=14923;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#170\170]8;;\

           INFO     X_rowsum.min(): -8.594079315662384               ]8;id=511868;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=772733;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#171\171]8;;\

           INFO     X_rowsum.max(): 8.860138446092606                ]8;id=752318;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=914283;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#172\172]8;;\

Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Alphas found: ['-0.0000', '0.1000', '0.1000', '0.1000', '0.1000', '-0.0000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.0109', '0.0000', '0.0000', '0.1000', '0.0721', '0.1000', '0.1000', '0.1000', '0.0000', '0.1000', '0.1000', '0.0972', '0.1000', '0.1000', '-0.0000', '0.0672', '0.1000', '0.1000', '-0.0000', '0.0642', '0.1000', '0.1000', '0.1000', '0.0000', '0.1000', '0.0173', '0.1000', '0.1000', '0.1000']
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_de

{'trn_acc': 0.925000011920929,
 'val_acc_l': [0.875, 0.9750000238418579],
 'test_acc': 0.8999999761581421,
 'trn_min_ypred': -2.261595931413054,
 'trn_max_ypred': 1.8336543988375213,
 'trn_min_ntkunlabeled': -0.6051960390713326,
 'trn_max_ntkunlabeled': 16.55578889740822,
 'val_svm_alphas': [tensor([-8.5413e-11,  1.0000e-01,  1.0000e-01,  1.0000e-01,  1.0000e-01,
          -1.1821e-11,  1.0000e-01,  1.0000e-01,  1.0000e-01,  1.0000e-01,
           1.0000e-01,  1.0950e-02,  1.2126e-12,  1.3593e-11,  1.0000e-01,
           7.2130e-02,  1.0000e-01,  1.0000e-01,  1.0000e-01,  3.0370e-11,
           1.0000e-01,  1.0000e-01,  9.7220e-02,  1.0000e-01,  1.0000e-01,
          -4.4990e-11,  6.7161e-02,  1.0000e-01,  1.0000e-01, -3.1906e-11,
           6.4171e-02,  1.0000e-01,  1.0000e-01,  1.0000e-01,  1.8713e-12,
           1.0000e-01,  1.7307e-02,  1.0000e-01,  1.0000e-01,  1.0000e-01],
         dtype=torch.float64),
  tensor([ 1.0000e-01,  1.0000e-01, -4.3520e-12,  1.0000e-01,  1.0796e-10,
  

In [24]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0, n_adversarial=0, C=0.01)

[22:09:04] INFO     Starting experiment            ]8;id=24646;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=646984;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#105\105]8;;\
                    exp_ntk_svm_hyperparam with                                 
                    configuration:                                              

           INFO     data_params: {'dataset':       ]8;id=736473;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=116133;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#106\106]8;;\
                    'csbm', 'learning_setting':                                 
                    'transductive',                                             
                    'specification': {'classes':                                
                    2, 'n_trn_labeled': 40,                                     
                    'n_trn_unlabeled': 0, 'n_val':                              
                    40, 'n_test': 120, 'sigma': 1,                              
                    'avg_within_class_degree':                                  
                    3.16,                                                       
                    'avg_between_class_degree':                                 
                    0.74, 'K': 1.5, 'seed': 0},                                 
                    'cv_folds': 2}                                              

           INFO     model_params: {'label': 'GCN', ]8;id=780755;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=3945;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#107\107]8;;\
                    'model': 'GCN',                                             
                    'normalization':                                            
                    'row_normalization', 'depth':                               
                    1, 'regularizer': 0.01,                                     
                    'pred_method': 'svm',                                       
                    'activation': 'relu',                                       
                    'cache_size': 10000, 'bias':                                
                    False, 'solver': 'qplayer',                                 
                    'alpha_tol': 0.0001}                                        

           INFO     verbosity_params:              ]8;id=499721;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=576704;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#108\108]8;;\
                    {'debug_lvl': 'info'}                                       

           INFO     other_params: {'device':       ]8;id=955286;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=132650;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#109\109]8;;\
                    'cpu', 'dtype': torch.float64,                              
                    'allow_tf32': False, 'debug':                               
                    True}                                                       

...

           INFO     X_rowsum.median(): -0.23559162765741348          ]8;id=515168;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=619210;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#170\170]8;;\

           INFO     X_rowsum.min(): -8.594079315662384               ]8;id=927711;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=867916;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#171\171]8;;\

           INFO     X_rowsum.max(): 8.860138446092606                ]8;id=480649;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=899683;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#172\172]8;;\

Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Alphas found: ['0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100']
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.mi

{'trn_acc': 0.887499988079071,
 'val_acc_l': [0.8500000238418579, 0.9750000238418579],
 'test_acc': 0.8833333253860474,
 'trn_min_ypred': -0.7998894021886204,
 'trn_max_ypred': 0.5134467528641486,
 'trn_min_ntkunlabeled': -0.6051960390713326,
 'trn_max_ntkunlabeled': 16.55578889740822,
 'val_svm_alphas': [tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100, 0.0100], dtype=torch.float64),
  tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100

In [25]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0, n_adversarial=0, C=1.5)

[22:09:34] INFO     Starting experiment            ]8;id=593370;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=425630;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#105\105]8;;\
                    exp_ntk_svm_hyperparam with                                 
                    configuration:                                              

           INFO     data_params: {'dataset':       ]8;id=574688;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=326964;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#106\106]8;;\
                    'csbm', 'learning_setting':                                 
                    'transductive',                                             
                    'specification': {'classes':                                
                    2, 'n_trn_labeled': 40,                                     
                    'n_trn_unlabeled': 0, 'n_val':                              
                    40, 'n_test': 120, 'sigma': 1,                              
                    'avg_within_class_degree':                                  
                    3.16,                                                       
                    'avg_between_class_degree':                                 
                    0.74, 'K': 1.5, 'seed': 0},                                 
                    'cv_folds': 2}                                              

           INFO     model_params: {'label': 'GCN', ]8;id=355700;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=592399;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#107\107]8;;\
                    'model': 'GCN',                                             
                    'normalization':                                            
                    'row_normalization', 'depth':                               
                    1, 'regularizer': 1.5,                                      
                    'pred_method': 'svm',                                       
                    'activation': 'relu',                                       
                    'cache_size': 10000, 'bias':                                
                    False, 'solver': 'qplayer',                                 
                    'alpha_tol': 0.0001}                                        

           INFO     verbosity_params:              ]8;id=891818;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=434848;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#108\108]8;;\
                    {'debug_lvl': 'info'}                                       

           INFO     other_params: {'device':       ]8;id=374869;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py\exp_ntk_svm_hyperparam.py]8;;\:]8;id=423558;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_svm_hyperparam.py#109\109]8;;\
                    'cpu', 'dtype': torch.float64,                              
                    'allow_tf32': False, 'debug':                               
                    True}                                                       

...

           INFO     X_rowsum.median(): -0.23559162765741348          ]8;id=490074;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=870428;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#170\170]8;;\

           INFO     X_rowsum.min(): -8.594079315662384               ]8;id=723585;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=179656;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#171\171]8;;\

           INFO     X_rowsum.max(): 8.860138446092606                ]8;id=344663;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=56467;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#172\172]8;;\

Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Alphas found: ['0.0000', '1.5000', '0.0000', '0.7218', '0.0000', '0.0000', '0.0000', '1.1951', '1.5000', '-0.0000', '0.0000', '0.0000', '-0.0000', '0.0000', '0.2816', '0.0000', '1.2398', '0.0000', '1.5000', '0.0000', '0.7259', '-0.0000', '0.0000', '0.0000', '1.3442', '0.0000', '0.0000', '0.0000', '0.8251', '0.0000', '0.0000', '0.0000', '1.5000', '-0.0000', '0.0000', '1.5000', '0.0000', '0.0000', '0.0000', '0.6542']
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_der.min(): 0.05078310731235101
E_der.max(): 1.0
Sig.mean(): 0.6320897987690165
Sig.min(): 0.06878726054202187
Sig.max(): 8.27789444870411
Sig.mean(): 0.022997129009389185
Sig.min(): -5.485176016107152
Sig.max(): 8.27789444870411
Depth 0
E_de

{'trn_acc': 0.9375,
 'val_acc_l': [0.8500000238418579, 0.8999999761581421],
 'test_acc': 0.8916666507720947,
 'trn_min_ypred': -4.577225849829949,
 'trn_max_ypred': 3.206486224582233,
 'trn_min_ntkunlabeled': -0.6051960390713326,
 'trn_max_ntkunlabeled': 16.55578889740822,
 'val_svm_alphas': [tensor([ 2.4119e-09,  1.5000e+00,  2.0742e-10,  7.2178e-01,  1.4527e-10,
           4.1416e-10,  2.5435e-10,  1.1951e+00,  1.5000e+00, -5.3074e-11,
           9.6225e-11,  4.0169e-10, -1.1716e-11,  3.1035e-10,  2.8163e-01,
           2.3041e-10,  1.2398e+00,  1.4102e-11,  1.5000e+00,  1.0652e-10,
           7.2591e-01, -2.1597e-11,  7.2119e-11,  4.9653e-11,  1.3442e+00,
           8.2335e-10,  1.5114e-10,  1.4848e-10,  8.2509e-01,  4.4531e-10,
           1.3162e-10,  3.6263e-11,  1.5000e+00, -7.8862e-11,  3.0168e-11,
           1.5000e+00,  6.0425e-11,  4.0876e-11,  3.5276e-11,  6.5417e-01],
         dtype=torch.float64),
  tensor([ 1.2974e+00,  1.5000e+00, -1.1990e-12,  1.6562e-12,  6.0848e-12,
 

In [6]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0, n_adversarial=0)

[13:09:36] INFO     Starting experiment exp_ntk_certify   ]8;id=987579;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=544350;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#116\116]8;;\
                    with configuration:                                         

           INFO     data_params: {'dataset': 'csbm',      ]8;id=26859;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=97635;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#117\117]8;;\
                    'learning_setting': 'transductive',                         
                    'specification': {'classes': 2,                             
                    'n_trn_labeled': 40,                                        
                    'n_trn_unlabeled': 0, 'n_val': 40,                          
                    'n_test': 120, 'sigma': 1,                                  
                    'avg_within_class_degree': 3.16,                            
                    'avg_between_class_degree': 0.74,                           
                    'K': 1.5, 'seed': 0}}                                       

           INFO     model_params: {'label': 'GCN',        ]8;id=159986;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=360586;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#118\118]8;;\
                    'model': 'GCN', 'normalization':                            
                    'row_normalization', 'depth': 1,                            
                    'regularizer': 1, 'pred_method':                            
                    'svm', 'activation': 'relu',                                
                    'cache_size': 10000, 'bias': False,                         
                    'solver': 'qplayer', 'alpha_tol':                           
                    0.0001}                                                     

           INFO     certification_params:                 ]8;id=646518;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=138292;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#119\119]8;;\
                    {'n_adversarial': 0,                                        
                    'perturbation_model': 'linf',                               
                    'delta': 0, 'method': 'XXT',                                
                    'cert_method': 'bilevel_svm'}                               

           INFO     verbosity_params: {'debug_lvl':       ]8;id=973050;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=78798;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#120\120]8;;\
                    'info'}                                                     

...

in callback  User MIP start did not produce a new incumbent solution

in callback  User MIP start violates constraint R12961 by 0.772707208

in callback  Another try with MIP start


Interrupt request received
optimization status  11
robust_count  5
tensor(-0.8035, dtype=torch.float64) False
Set parameter IntegralityFocus to value 1


[13:10:33] INFO     Set parameter IntegralityFocus to value 1       ]8;id=375922;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py\utils.py]8;;\:]8;id=924385;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py#240\240]8;;\

Set parameter IntFeasTol to value 0.0001


           INFO     Set parameter IntFeasTol to value 0.0001        ]8;id=791956;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py\utils.py]8;;\:]8;id=918095;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py#241\241]8;;\

in callback  User MIP start did not produce a new incumbent solution

in callback  User MIP start violates constraint R12961 by 0.772707208

in callback  Another try with MIP start



In [7]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0.1, n_adversarial=10, C=0.01)

[13:20:26] INFO     Starting experiment exp_ntk_certify   ]8;id=383249;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=857279;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#116\116]8;;\
                    with configuration:                                         

           INFO     data_params: {'dataset': 'csbm',      ]8;id=906412;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=508726;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#117\117]8;;\
                    'learning_setting': 'transductive',                         
                    'specification': {'classes': 2,                             
                    'n_trn_labeled': 40,                                        
                    'n_trn_unlabeled': 0, 'n_val': 40,                          
                    'n_test': 120, 'sigma': 1,                                  
                    'avg_within_class_degree': 3.16,                            
                    'avg_between_class_degree': 0.74,                           
                    'K': 1.5, 'seed': 0}}                                       

           INFO     model_params: {'label': 'GCN',        ]8;id=206985;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=570217;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#118\118]8;;\
                    'model': 'GCN', 'normalization':                            
                    'row_normalization', 'depth': 1,                            
                    'regularizer': 0.01, 'pred_method':                         
                    'svm', 'activation': 'relu',                                
                    'cache_size': 10000, 'bias': False,                         
                    'solver': 'qplayer', 'alpha_tol':                           
                    0.0001}                                                     

           INFO     certification_params:                 ]8;id=778313;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=46272;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#119\119]8;;\
                    {'n_adversarial': 10,                                       
                    'perturbation_model': 'linf',                               
                    'delta': 0.1, 'method': 'XXT',                              
                    'cert_method': 'bilevel_svm'}                               

           INFO     verbosity_params: {'debug_lvl':       ]8;id=716924;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=238166;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#120\120]8;;\
                    'info'}                                                     

...

Set parameter IntFeasTol to value 0.0001


           INFO     Set parameter IntFeasTol to value 0.0001        ]8;id=593119;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py\utils.py]8;;\:]8;id=457264;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py#244\244]8;;\

in callback  User MIP start did not produce a new incumbent solution

in callback  User MIP start violates constraint R12960 by 0.227535035

optimization status  2
Original  tensor(0.1880, dtype=torch.float64)
Obj: 0.0664995
robust_count  105


           INFO     Accuracy 0.8833333253860474           ]8;id=18236;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=803712;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#259\259]8;;\

{'accuracy': 0.8833333253860474,
 'accuracy_ub': 0.8666666746139526,
 'accuracy_lb': 0.875,
 'accuracy_cert_trivial': 0.425,
 'accuracy_cert': 0.875,
 'accuracy_cert_unrobust': 0,
 'min_ypred': -1.025991512076073,
 'max_ypred': 0.4773363470294938,
 'min_ylb': -1.1017231032701087,
 'max_ylb': 0.42010552255278066,
 'min_yub': -0.9617346131443026,
 'max_yub': 0.6849280312342676,
 'avg_ntklb': 0.661336246087787,
 'min_ntklb': -0.6418988081434144,
 'max_ntklb': 16.55578889740822,
 'avg_ntkub': 0.8865976655689608,
 'min_ntkub': -0.6418988081434144,
 'max_ntkub': 16.55578889740822,
 'min_ntklabeled': -0.5145129741624762,
 'max_ntklabeled': 16.565788897408222,
 'avg_ntkunlabeled': 0.7500743714661756,
 'min_ntkunlabeled': -0.607958364783971,
 'max_ntkunlabeled': 9.249320453253077,
 'cond': 8520.478798011947}

In [8]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0.1, n_adversarial=50, C=0.01)

[13:24:17] INFO     Starting experiment exp_ntk_certify   ]8;id=578706;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=159732;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#116\116]8;;\
                    with configuration:                                         

           INFO     data_params: {'dataset': 'csbm',      ]8;id=975106;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=337622;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#117\117]8;;\
                    'learning_setting': 'transductive',                         
                    'specification': {'classes': 2,                             
                    'n_trn_labeled': 40,                                        
                    'n_trn_unlabeled': 0, 'n_val': 40,                          
                    'n_test': 120, 'sigma': 1,                                  
                    'avg_within_class_degree': 3.16,                            
                    'avg_between_class_degree': 0.74,                           
                    'K': 1.5, 'seed': 0}}                                       

           INFO     model_params: {'label': 'GCN',        ]8;id=837226;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=936193;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#118\118]8;;\
                    'model': 'GCN', 'normalization':                            
                    'row_normalization', 'depth': 1,                            
                    'regularizer': 0.01, 'pred_method':                         
                    'svm', 'activation': 'relu',                                
                    'cache_size': 10000, 'bias': False,                         
                    'solver': 'qplayer', 'alpha_tol':                           
                    0.0001}                                                     

           INFO     certification_params:                 ]8;id=287782;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=675609;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#119\119]8;;\
                    {'n_adversarial': 50,                                       
                    'perturbation_model': 'linf',                               
                    'delta': 0.1, 'method': 'XXT',                              
                    'cert_method': 'bilevel_svm'}                               

           INFO     verbosity_params: {'debug_lvl':       ]8;id=537455;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=31701;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#120\120]8;;\
                    'info'}                                                     

...

Set parameter IntFeasTol to value 0.0001


           INFO     Set parameter IntFeasTol to value 0.0001        ]8;id=418713;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py\utils.py]8;;\:]8;id=313231;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py#244\244]8;;\

in callback  User MIP start did not produce a new incumbent solution

in callback  User MIP start violates constraint R12960 by 0.227535035

optimization status  2
Original  tensor(0.1880, dtype=torch.float64)
Obj: -0.30051
robust_count  14


           INFO     Accuracy 0.8833333253860474           ]8;id=645268;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=114230;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#259\259]8;;\

{'accuracy': 0.8833333253860474,
 'accuracy_ub': 0.675000011920929,
 'accuracy_lb': 0.42500001192092896,
 'accuracy_cert_trivial': 0.06666666666666667,
 'accuracy_cert': 0.11666666666666667,
 'accuracy_cert_unrobust': 0,
 'min_ypred': -1.025991512076073,
 'max_ypred': 0.4773363470294938,
 'min_ylb': -1.6703329258839115,
 'max_ylb': 0.21773690641644441,
 'min_yub': -0.7278578283809487,
 'max_yub': 1.2875013934211537,
 'avg_ntklb': 0.2510536384311264,
 'min_ntklb': -1.2224336620721192,
 'max_ntklb': 16.55578889740822,
 'avg_ntkub': 1.5533132638031681,
 'min_ntkub': -0.6418988081434144,
 'max_ntkub': 16.55578889740822,
 'min_ntklabeled': -0.5145129741624762,
 'max_ntklabeled': 16.565788897408222,
 'avg_ntkunlabeled': 0.7500743714661756,
 'min_ntkunlabeled': -0.607958364783971,
 'max_ntkunlabeled': 9.249320453253077,
 'cond': 8520.478798011947}

In [7]:
validate_bilevel_svm(n_trn_labeled=10, n_val=10, n_test=5, delta=0, n_adversarial=0, C=0.1)

[14:58:24] INFO     Starting experiment exp_ntk_certify   ]8;id=296781;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=358501;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#116\116]8;;\
                    with configuration:                                         

           INFO     data_params: {'dataset': 'csbm',      ]8;id=919592;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=360685;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#117\117]8;;\
                    'learning_setting': 'transductive',                         
                    'specification': {'classes': 2,                             
                    'n_trn_labeled': 10,                                        
                    'n_trn_unlabeled': 0, 'n_val': 10,                          
                    'n_test': 5, 'sigma': 1,                                    
                    'avg_within_class_degree': 3.16,                            
                    'avg_between_class_degree': 0.74,                           
                    'K': 1.5, 'seed': 0}}                                       

           INFO     model_params: {'label': 'GCN',        ]8;id=774597;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=420636;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#118\118]8;;\
                    'model': 'GCN', 'normalization':                            
                    'row_normalization', 'depth': 1,                            
                    'regularizer': 0.1, 'pred_method':                          
                    'svm', 'activation': 'relu',                                
                    'cache_size': 10000, 'bias': False,                         
                    'solver': 'cvxopt', 'alpha_tol':                            
                    0.0001}                                                     

           INFO     certification_params:                 ]8;id=691526;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=478409;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#119\119]8;;\
                    {'n_adversarial': 0,                                        
                    'perturbation_model': 'linf',                               
                    'delta': 0, 'method': 'XXT',                                
                    'cert_method': 'bilevel_svm'}                               

           INFO     verbosity_params: {'debug_lvl':       ]8;id=10545;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=920427;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#120\120]8;;\
                    'info'}                                                     

...

Set parameter IntFeasTol to value 0.0001


           INFO     Set parameter IntFeasTol to value 0.0001        ]8;id=574127;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py\utils.py]8;;\:]8;id=201537;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py#239\239]8;;\

in callback  User MIP start did not produce a new incumbent solution

in callback  Another try with MIP start

optimization status  10
Original  tensor(1.9856, dtype=torch.float64)
Obj: 0.989669
robust_count  6


           INFO     Accuracy 1.0                          ]8;id=409503;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=366477;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#254\254]8;;\

{'accuracy': 1.0,
 'accuracy_ub': 1.0,
 'accuracy_lb': 1.0,
 'accuracy_cert_trivial': 1.0,
 'accuracy_cert': 1.0,
 'accuracy_cert_unrobust': 0,
 'min_ypred': 1.2362267438107097,
 'max_ypred': 2.1879724195791552,
 'min_ylb': 1.2362267438107097,
 'max_ylb': 2.1879724195791552,
 'min_yub': 1.2362267438107097,
 'max_yub': 2.1879724195791552,
 'avg_ntklb': 1.1640829420207255,
 'min_ntklb': -0.10799942291542018,
 'max_ntklb': 3.0529409694550624,
 'avg_ntkub': 1.1640829420207255,
 'min_ntkub': -0.10799942291542018,
 'max_ntkub': 3.0529409694550624,
 'min_ntklabeled': -0.08004117964150392,
 'max_ntklabeled': 3.1529409694550625,
 'avg_ntkunlabeled': 1.3428311518927516,
 'min_ntkunlabeled': -0.10799942291542018,
 'max_ntkunlabeled': 2.9463874146883633,
 'cond': 235.08848360585142}

In [11]:
validate_bilevel_svm(n_trn_labeled=40, n_val=40, n_test=120, delta=0, n_adversarial=0)

[14:14:58] INFO     Starting experiment exp_ntk_certify   ]8;id=522449;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=573499;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#116\116]8;;\
                    with configuration:                                         

           INFO     data_params: {'dataset': 'csbm',      ]8;id=689911;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=387284;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#117\117]8;;\
                    'learning_setting': 'transductive',                         
                    'specification': {'classes': 2,                             
                    'n_trn_labeled': 40,                                        
                    'n_trn_unlabeled': 0, 'n_val': 40,                          
                    'n_test': 120, 'sigma': 1,                                  
                    'avg_within_class_degree': 3.16,                            
                    'avg_between_class_degree': 0.74,                           
                    'K': 1.5, 'seed': 0}}                                       

           INFO     model_params: {'label': 'GCN',        ]8;id=356551;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=865131;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#118\118]8;;\
                    'model': 'GCN', 'normalization':                            
                    'row_normalization', 'depth': 1,                            
                    'regularizer': 1, 'pred_method':                            
                    'svm', 'activation': 'relu',                                
                    'cache_size': 10000, 'bias': False,                         
                    'solver': 'cvxopt', 'alpha_tol':                            
                    0.0001}                                                     

           INFO     certification_params:                 ]8;id=511189;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=741004;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#119\119]8;;\
                    {'n_adversarial': 0,                                        
                    'perturbation_model': 'linf',                               
                    'delta': 0, 'method': 'XXT',                                
                    'cert_method': 'bilevel_svm'}                               

           INFO     verbosity_params: {'debug_lvl':       ]8;id=424321;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=172915;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#120\120]8;;\
                    'info'}                                                     

...

Set parameter IntFeasTol to value 0.0001


           INFO     Set parameter IntFeasTol to value 0.0001        ]8;id=670844;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py\utils.py]8;;\:]8;id=953331;file:///ceph/ssd/students/sabanaya/ntk-robust/src/utils.py#239\239]8;;\

Error code 10010: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information


           INFO     Accuracy 0.875                        ]8;id=996111;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=619687;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#254\254]8;;\

{'accuracy': 0.875,
 'accuracy_ub': 0.875,
 'accuracy_lb': 0.875,
 'accuracy_cert_trivial': 1.0,
 'accuracy_cert': None,
 'accuracy_cert_unrobust': 0,
 'min_ypred': -95.16589364011656,
 'max_ypred': 37.03006893352396,
 'min_ylb': -95.16589364011656,
 'max_ylb': 37.030068933523964,
 'min_yub': -95.16589364011656,
 'max_yub': 37.030068933523964,
 'avg_ntklb': 0.7608179200655608,
 'min_ntklb': -0.6418988081434144,
 'max_ntklb': 16.55578889740822,
 'avg_ntkub': 0.7608179200655608,
 'min_ntkub': -0.6418988081434144,
 'max_ntkub': 16.55578889740822,
 'min_ntklabeled': -0.5145129741624762,
 'max_ntklabeled': 17.55578889740822,
 'avg_ntkunlabeled': 0.7500743714661756,
 'min_ntkunlabeled': -0.607958364783971,
 'max_ntkunlabeled': 9.249320453253077,
 'cond': 113.69188361056523}

# old stuff

In [5]:
def run_exp(n_seeds, data_params, model_params, certificate_params,
            verbosity_params, other_params):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    for seed in range(n_seeds):
        data_params["specification"]["seed"] = seed
        res = run(data_params, model_params, certificate_params,
                  verbosity_params, other_params, seed)
        acc_l.append(res["accuracy"])
        min_ypred.append(res["min_ypred"])
        max_ypred.append(res["max_ypred"])
        min_ntklabeled.append(res["min_ntklabeled"])
        max_ntklabeled.append(res["max_ntklabeled"])
        min_ntkunlabeled.append(res["min_ntkunlabeled"])
        max_ntkunlabeled.append(res["max_ntkunlabeled"])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [16]:
model_params["regularizer"] = 1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "csbm"
data_params["specification"]["avg_within_class_degree"] = 1.58 * 2
data_params["specification"]["avg_between_class_degree"] = 0.37 * 2
other_params["device"] = 0
certificate_params["n_adversarial"] = 50
certificate_params["delta"] = 0.1
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.005
model_params["solver"] = "sklearn"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = True
data_params["learning_setting"] = "transductive"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

[04/09/24 22:45:51] INFO     Starting experiment exp_ntk_certify with configuration:         ]8;id=933882;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=515844;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#116\116]8;;\

                    INFO     data_params: {'dataset': 'csbm', 'learning_setting':            ]8;id=244099;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=966482;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#117\117]8;;\
                             'transductive', 'specification': {'classes': 2,                                       
                             'n_trn_labeled': 20, 'n_trn_unlabeled': 0, 'n_val': 20,                               
                             'n_test': 500, 'sigma': 1, 'avg_within_class_degree': 3.16,                           
                             'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 0}}                               

                    INFO     model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': ]8;id=682435;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=983295;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#118\118]8;;\
                             'row_normalization', 'depth': 1, 'regularizer': 0.005,                                
                             'pred_method': 'svm', 'cache_size': 10000, 'solver': 'sklearn',                       
                             'alpha_tol': 0.0001, 'bias': True}                                                    

                    INFO     certification_params: {'n_adversarial': 50,                     ]8;id=564483;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=221776;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#119\119]8;;\
                             'perturbation_model': 'linf', 'delta': 0.1, 'method': 'XXT'}                          

                    INFO     verbosity_params: {'debug_lvl': 'info'}                         ]8;id=622213;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=647776;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#120\120]8;;\

...

                    INFO     Currently on gpu device cuda:0                                  ]8;id=661557;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py\exp_ntk_certify.py]8;;\:]8;id=176458;file:///ceph/ssd/students/sabanaya/ntk-robust/exp_ntk_certify.py#152\152]8;;\

CSBM mu:
[0.20044594 0.20044594 0.20044594 0.20044594 0.20044594 0.20044594
 0.20044594 0.20044594 0.20044594 0.20044594 0.20044594 0.20044594
 0.20044594 0.20044594]


                    INFO     CSBM(p=0.01173, q=0.00275)                                                  ]8;id=444740;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py\data.py]8;;\:]8;id=386578;file:///ceph/ssd/students/sabanaya/ntk-robust/src/data.py#26\26]8;;\

Alphas found: ['-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '-0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050', '0.0050']


AssertionError: 

In [21]:
model_params["regularizer"] = 1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "csbm"
data_params["specification"]["avg_within_class_degree"] = 1.58 * 2
data_params["specification"]["avg_between_class_degree"] = 0.37 * 2
other_params["device"] = 0
certificate_params["n_adversarial"] = 50
certificate_params["delta"] = 0.01
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-02-12 09:26:51 (INFO): Starting experiment exp_ntk_certify with configuration:
2024-02-12 09:26:51 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'inductive', 'specification': {'classes': 2, 'n_trn_labeled': 40, 'n_trn_unlabeled': 0, 'n_val': 40, 'n_test': 1000, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 0}}
2024-02-12 09:26:51 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 1, 'pred_method': 'svm', 'cache_size': 10000}
2024-02-12 09:26:51 (INFO): certification_params: {'n_adversarial': 50, 'perturbation_model': 'linf', 'delta': 0.01, 'method': 'XXT'}
2024-02-12 09:26:51 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-02-12 09:26:51 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-02-12 09:26:51 (INFO): seed: 0
2024-02-12 09:26:51 (INFO): Currently on gpu device cuda:0
2024-02-12 09:26:51 (INFO): CSBM(

CSBM mu:
[0.15990053 0.15990053 0.15990053 0.15990053 0.15990053 0.15990053
 0.15990053 0.15990053 0.15990053 0.15990053 0.15990053 0.15990053
 0.15990053 0.15990053 0.15990053 0.15990053 0.15990053 0.15990053
 0.15990053 0.15990053 0.15990053 0.15990053]


2024-02-12 09:26:51 (INFO): X.min(): -4.334216594696045
2024-02-12 09:26:51 (INFO): X.max(): 4.0621466636657715
2024-02-12 09:26:51 (INFO): X_rowsum.mean(): 0.3242250910269899
2024-02-12 09:26:51 (INFO): X_rowsum.median(): 0.5348266516812146
2024-02-12 09:26:51 (INFO): X_rowsum.min(): -16.154811039566994
2024-02-12 09:26:51 (INFO): X_rowsum.max(): 16.045364283025265


X.mean() 0.01473750413759045
Sig.mean(): 0.3020178033977416
Sig.min(): -15.339332392283062
Sig.max(): 41.04509567162934
Depth 0
E_der.min(): 0.25584292500065875
E_der.max(): 0.9998576474901678
Sig.mean(): 5.7940700722308645
Sig.min(): 0.8994185947581148
Sig.max(): 41.043384333896
Sig.mean(): 0.022876857319693637
Sig.min(): -13.002585609908412
Sig.max(): 33.790778429342936
Depth 0
E_der.min(): 0.1868783171694024
E_der.max(): 0.9998576474901678
Sig.mean(): 1.5804276973361686
Sig.min(): 0.47511456913163097
Sig.max(): 33.78936955373383
Sig.mean(): 0.022876857319693637
Sig.min(): -13.002585609908412
Sig.max(): 33.790778429342936
Sig[:,idx_adv]: 0.03456422411241992
Sig_ub.mean(): 0.03923710965717605
Sig_ub.min(): -13.002585609908412
Sig_ub.max(): 33.790778429342936
Sig_ub[:,idx_adv]: 0.08958288651516681
Sig_lb.mean(): 0.00651660498221123
Sig_lb.min(): -13.002585609908412
Sig_lb.max(): 33.790778429342936
Sig_lb[:,idx_adv]: -0.020454438290326973


2024-02-12 09:26:52 (INFO): Accuracy 0.8560000658035278


Depth 0
tensor(1.0000, device='cuda:0', dtype=torch.float64)
tensor(0., device='cuda:0', dtype=torch.float64)
tensor(1.0000, device='cuda:0', dtype=torch.float64)
tensor(-0.8326, device='cuda:0', dtype=torch.float64)
E_der.mean(): 0.502010636851994
E_der.min(): 0.1868783171694024
E_der.max(): 0.9998576474901678
E_der[:,idx_adv]: 0.5029711868060217
E_der_lb.mean(): 0.5005666238852353
E_der_lb.min(): 0.18687832839928578
E_der_lb.max(): 0.9998576474901678
E_der_lb[:,idx_adv]: 0.4982162919922147
E_der_ub.mean(): 0.5034406256576238
E_der_ub.min(): 0.1868783171694024
E_der_ub.max(): 1.0
E_der_ub[:,idx_adv]: 0.507653212476487
Sig.mean(): 1.5804276973361686
Sig.min(): 0.47511456913163097
Sig.max(): 33.78936955373383
Sig[:,idx_adv]: 1.5319346181972273
Sig_lb.mean(): 1.564269298924912
Sig_lb.min(): 0.4751145853961245
Sig_lb.max(): 33.789468865230404
Sig_lb[:,idx_adv]: 1.4816531671945996
Sig_ub.mean(): 1.5969449324469653
Sig_ub.min(): 0.47511457982371796
Sig_ub.max(): 33.78929879643642
Sig_ub[:,i

{'accuracy': 0.8560000658035278,
 'accuracy_ub': 0.859000027179718,
 'accuracy_lb': 0.8550000190734863,
 'accuracy_cert_trivial': 0.406,
 'accuracy_cert': 0.9140000343322754,
 'accuracy_cert_unrobust': 0.0,
 'min_ypred': -4.273266330950973,
 'max_ypred': 3.8556140409761834,
 'min_ylb': -4.382800222290024,
 'max_ylb': 3.762538402737156,
 'min_yub': -4.163358872577893,
 'max_yub': 3.9500315531399712,
 'avg_ntklb': 1.6490275346642045,
 'min_ntklb': -1.7130419420507819,
 'max_ntklb': 67.57543709245473,
 'avg_ntkub': 1.6987334378122962,
 'min_ntkub': -1.7130419136088704,
 'max_ntkub': 67.5751963663489,
 'min_ntklabeled': -2.7836303410421457,
 'max_ntklabeled': 83.08263713314018,
 'avg_ntkunlabeled': 1.619156476420116,
 'min_ntkunlabeled': -0.964955620036918,
 'max_ntkunlabeled': 13.331564131697945,
 'cond': 544.1500695342672}

In [10]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "csbm"
data_params["specification"]["avg_within_class_degree"] = 1.58 * 2
data_params["specification"]["avg_between_class_degree"] = 0.37 * 2
other_params["device"] = 0
certificate_params["n_adversarial"] = 50
certificate_params["delta"] = 0.005 #0.32
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-01-18 18:30:36 (INFO): Starting experiment exp_ntk_certify with configuration:
2024-01-18 18:30:36 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'inductive', 'specification': {'classes': 2, 'n_trn_labeled': 600, 'n_trn_unlabeled': 0, 'n_val': 200, 'n_test': 200, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 0}}
2024-01-18 18:30:36 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'cache_size': 10000}
2024-01-18 18:30:36 (INFO): certification_params: {'n_adversarial': 50, 'perturbation_model': 'linf', 'delta': 0.005, 'method': 'XXT'}
2024-01-18 18:30:36 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-01-18 18:30:36 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-01-18 18:30:36 (INFO): seed: 0
2024-01-18 18:30:36 (INFO): Currently on gpu device cuda:0
2024-01-18 18:30:36 (INFO): C

CSBM mu:
[0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342]


2024-01-18 18:30:36 (INFO): X.min(): -3.859495162963867
2024-01-18 18:30:36 (INFO): X.max(): 4.109212875366211
2024-01-18 18:30:36 (INFO): X_rowsum.mean(): 0.33397936742261686
2024-01-18 18:30:36 (INFO): X_rowsum.median(): 0.7591041489504278
2024-01-18 18:30:36 (INFO): X_rowsum.min(): -16.377630546689034
2024-01-18 18:30:36 (INFO): X_rowsum.max(): 17.149057352915406


X.mean() 0.015903779401076994
Sig.mean(): 0.030798085060794136
Sig.min(): -14.949451596673214
Sig.max(): 32.95754895220645
Depth 0
E_der.min(): 0.18289396597227744
E_der.max(): 0.9998576474901678
Sig.mean(): 1.8967366306355231
Sig.min(): 0.3569972147566615
Sig.max(): 32.95617481733851
Sig.mean(): 0.025097406718435484
Sig.min(): -14.462656274780015
Sig.max(): 31.56303964237527
Depth 0
E_der.min(): 0.18234846643512442
E_der.max(): 0.9998576474901678
Sig.mean(): 1.5459834786357913
Sig.min(): 0.44621182065889875
Sig.max(): 31.561723650297893
Sig.mean(): 0.025097406718435484
Sig.min(): -14.462656274780015
Sig.max(): 31.56303964237527
Sig[:,idx_adv]: 0.02448061310299977
Sig_ub.mean(): 0.033608742782804575
Sig_ub.min(): -14.462656274780015
Sig_ub.max(): 31.56303964237527
Sig_ub[:,idx_adv]: 0.05375579848451701
Sig_lb.mean(): 0.01658607065406639
Sig_lb.min(): -14.462656274780015
Sig_lb.max(): 31.56303964237527
Sig_lb[:,idx_adv]: -0.004794572278517465
Depth 0


2024-01-18 18:30:36 (INFO): Accuracy 0.8399999737739563


tensor(1.0000, device='cuda:0', dtype=torch.float64)
tensor(0., device='cuda:0', dtype=torch.float64)
tensor(1.0000, device='cuda:0', dtype=torch.float64)
tensor(-0.8665, device='cuda:0', dtype=torch.float64)
E_der.mean(): 0.5020980204544404
E_der.min(): 0.18234846643512442
E_der.max(): 0.9998576474901678
E_der[:,idx_adv]: 0.5024218047014748
E_der_lb.mean(): 0.5013197362026182
E_der_lb.min(): 0.1663939765381661
E_der_lb.max(): 0.9998576474901678
E_der_lb[:,idx_adv]: 0.4999765824759392
E_der_ub.mean(): 0.5028634999968823
E_der_ub.min(): 0.1968149039623989
E_der_ub.max(): 1.0
E_der_ub[:,idx_adv]: 0.5048062853550392
Sig.mean(): 1.5459834786357913
Sig.min(): 0.44621182065889875
Sig.max(): 31.561723650297893
Sig[:,idx_adv]: 1.5177168363382225
Sig_lb.mean(): 1.53755551367147
Sig_lb.min(): 0.44773812324844975
Sig_lb.max(): 31.561822862760184
Sig_lb[:,idx_adv]: 1.492128733554037
Sig_ub.mean(): 1.5545058656373965
Sig_ub.min(): 0.4446740196944931
Sig_ub.max(): 31.561652929061356
Sig_ub[:,idx_adv

{'accuracy': 0.8399999737739563,
 'accuracy_ub': 0.85999995470047,
 'accuracy_lb': 0.7849999666213989,
 'accuracy_cert_trivial': 0.305,
 'accuracy_cert': 0.8349999785423279,
 'accuracy_cert_unrobust': 0.0,
 'min_ypred': -5.780266064856437,
 'max_ypred': 4.81409235565413,
 'min_ylb': -8.65290782927396,
 'max_ylb': 4.6379661741879294,
 'min_yub': -5.621347871695073,
 'max_yub': 5.49149230855254,
 'avg_ntklb': 1.6295574933412396,
 'min_ntklb': -2.204459440184935,
 'max_ntklb': 63.12036942722443,
 'avg_ntkub': 1.6552940009899948,
 'min_ntkub': -2.2044594244878803,
 'max_ntkub': 63.120128872274606,
 'min_ntklabeled': -2.3270710965361756,
 'max_ntklabeled': 66.00903217973368,
 'avg_ntkunlabeled': 1.6436886444665464,
 'min_ntkunlabeled': -2.2044594382782496,
 'max_ntkunlabeled': 26.982307255502207,
 'cond': 17377.589908277496}

In [21]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "cora"
other_params["device"] = 0
certificate_params["n_adversarial"] = 50
certificate_params["delta"] = 10
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-01-16 11:16:40 (INFO): Starting experiment exp_ntk_certify with configuration:
2024-01-16 11:16:40 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'inductive', 'specification': {'n_per_class': 20, 'fraction_test': 0.1, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True, 'seed': 0}}
2024-01-16 11:16:40 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'cache_size': 10000}
2024-01-16 11:16:40 (INFO): certification_params: {'n_adversarial': 50, 'perturbation_model': 'l0', 'delta': 10}
2024-01-16 11:16:40 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-01-16 11:16:40 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-01-16 11:16:40 (INFO): seed: 0
2024-01-16 11:16:40 (INFO): Currently on gpu device cuda:0
2024-01-16 11:16:40 (INFO): X.min(): 0.0
2024-01-16 11:16:40 (INFO): X.max(): 1.0
2024-01-16 11

E_der.min(): 0.4999999681690114
E_der.max(): 0.9998576474901678
Sig.min(): 0.5152510432128604
Sig.max(): 26.998874259688176
E_der.min(): 0.4999999681690114
E_der.max(): 0.9998576474901678
Sig.min(): 0.5152510432128604
Sig.max(): 16.499312047587217
tensor(1.0438, device='cuda:0', dtype=torch.float64)
tensor(0., device='cuda:0', dtype=torch.float64)
tensor(30., device='cuda:0', dtype=torch.float64)
tensor(1.0066, device='cuda:0', dtype=torch.float64)
tensor(0., device='cuda:0', dtype=torch.float64)
tensor(30., device='cuda:0', dtype=torch.float64)
tensor(1.3910, device='cuda:0', dtype=torch.float64)
tensor(0., device='cuda:0', dtype=torch.float64)
tensor(44., device='cuda:0', dtype=torch.float64)
Adversarial Nodes: 
tensor(1.0180, device='cuda:0', dtype=torch.float64)
tensor(0.0006, device='cuda:0', dtype=torch.float64)
tensor(10.5960, device='cuda:0', dtype=torch.float64)
E_der_lb.min(): 0.4999999681690114
E_der_lb.max(): 0.9998576474901678
E_der_ub.min(): 0.4999999681690114
E_der_ub.ma

2024-01-16 11:16:44 (INFO): Accuracy 0.8498168587684631


{'accuracy': 0.8498168587684631,
 'accuracy_ub': 0.692307710647583,
 'accuracy_lb': 0.5018315315246582,
 'accuracy_cert_trivial': 0.5018315018315018,
 'accuracy_cert_trivial_n3': 0.26373626373626374,
 'accuracy_cert': 0.0,
 'accuracy_cert_unrobust': 0.0,
 'min_ypred': -1.7300217151641846,
 'max_ypred': 1.760793685913086,
 'min_ylb': -87.77947235107422,
 'max_ylb': 0.4492095112800598,
 'min_yub': -0.9994264245033264,
 'max_yub': 85.43028259277344,
 'min_ntklb': 0.04929542278416521,
 'max_ntklb': 32.997061111290144,
 'min_ntkub': 0.5152510432128604,
 'max_ntkub': 115.07919422632553,
 'min_ntklabeled': 1.0913821372767933,
 'max_ntklabeled': 50.09539883511362,
 'min_ntkunlabeled': 0.905161675184342,
 'max_ntkunlabeled': 24.99769941755681,
 'cond': 9640.409128523965}

In [40]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "cora_inv"
other_params["device"] = 0
certificate_params["n_adversarial"] = 20
certificate_params["delta"] = 1
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-01-16 15:47:00 (INFO): Starting experiment exp_ntk_certify with configuration:
2024-01-16 15:47:00 (INFO): data_params: {'dataset': 'cora_inv', 'learning_setting': 'inductive', 'specification': {'n_per_class': 20, 'fraction_test': 0.1, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True, 'seed': 0}}
2024-01-16 15:47:00 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'cache_size': 10000}
2024-01-16 15:47:00 (INFO): certification_params: {'n_adversarial': 20, 'perturbation_model': 'l0', 'delta': 1}
2024-01-16 15:47:00 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-01-16 15:47:00 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-01-16 15:47:00 (INFO): seed: 0
2024-01-16 15:47:00 (INFO): Currently on gpu device cuda:0
2024-01-16 15:47:00 (INFO): X.min(): -0.0
2024-01-16 15:47:00 (INFO): X.max(): 1.0
2024-01-1

Sig.mean(): 1397.745263856452
Sig.min(): 1379.0
Sig.max(): 1431.0
Depth 0
E_der.min(): 0.9375182858193654
E_der.max(): 0.9998576474901678
Sig.mean(): 1397.8480488854593
Sig.min(): 1380.1238343626485
Sig.max(): 1430.9403357634733
Sig.mean(): 1397.7161437246793
Sig.min(): 1380.75
Sig.max(): 1429.25
Depth 0
E_der.min(): 0.9476692472766347
E_der.max(): 0.9998576474903258
Sig.mean(): 1397.800646147605
Sig.min(): 1384.1845729538495
Sig.max(): 1428.6904295751658
tensor([1.6384, 1.0137, 1.2899, 1.4631, 1.1035, 0.6195, 0.9279, 0.0171, 0.5131,
        0.3474, 0.4104, 2.3155, 1.1784, 1.2134, 0.6100, 0.6753, 0.7357, 0.4146,
        1.3909, 0.5908, 1.7504, 0.5659, 0.7935, 1.4173, 0.7775, 0.9575, 1.6010,
        0.2632, 0.2647, 0.6792, 0.9212, 0.6438, 0.6959, 1.1539, 0.5482, 0.5096,
        0.3535, 0.2659, 0.6931, 0.9470, 1.0818, 1.2659, 0.3832, 0.9883, 0.9342,
        0.6362, 0.6489, 1.1349, 0.2571, 0.5823, 0.4549, 0.6565, 0.7382, 0.8070,
        0.2749, 0.7034, 0.6215, 0.7788, 0.6766, 0.3980, 1.01

2024-01-16 15:47:04 (INFO): Accuracy 0.8021978139877319


{'accuracy': 0.8021978139877319,
 'accuracy_ub': 0.7069597244262695,
 'accuracy_lb': 0.6190476417541504,
 'accuracy_cert_trivial': 0.7985347985347986,
 'accuracy_cert_trivial_n3': 0.4981684981684982,
 'accuracy_cert': 0.3333333432674408,
 'accuracy_cert_unrobust': 0.0,
 'min_ypred': -2.5327422618865967,
 'max_ypred': 2.953857898712158,
 'min_ylb': -57.691864013671875,
 'max_ylb': 2.4789297580718994,
 'min_yub': -1.8477323055267334,
 'max_yub': 53.8522834777832,
 'avg_ntklb': 2757.4826883125347,
 'min_ntklb': 2707.2181342848626,
 'max_ntklb': 2857.237144150098,
 'avg_ntkub': 2758.301069351067,
 'min_ntkub': 2707.2181334070156,
 'max_ntkub': 2857.2369009989516,
 'min_ntklabeled': 2693.0595892258734,
 'max_ntklabeled': 2854.337319496636,
 'avg_ntkunlabeled': 2756.862736557583,
 'min_ntkunlabeled': 2707.2181339577623,
 'max_ntkunlabeled': 2825.7055048558123,
 'cond': 7715100.963727744}

# SVM

In [ ]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

In [ ]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora"
model_params["pred_method"] = "svm"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

In [ ]:
model_params["regularizer"] = 1
data_params["dataset"] = "cora"
model_params["pred_method"] = "krr"
n_seeds = 2
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)